In [36]:
# Install required libraries
!pip install -q accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub gradio pandas

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [37]:
# Import necessary libraries
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer as LlamaTokenizer
from huggingface_hub import login
import gradio as gr
from textblob import TextBlob
import re

In [38]:
from google.colab import userdata

login(token=userdata.get('huggingface'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [46]:
llama_model_id = "meta-llama/Llama-3.2-3B-Instruct"
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_id, torch_dtype=torch.float16, device_map="auto")
llama_tokenizer = LlamaTokenizer.from_pretrained(llama_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
# Use device 0 for GPU if available, otherwise default to CPU
device = 0 if torch.cuda.is_available() else -1
device

0

In [48]:
# Initialize RoBERTa model for advanced sentiment analysis
roberta_model_id = "cardiffnlp/twitter-roberta-base-sentiment-latest"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_id)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_id)
sentiment_pipeline = pipeline("sentiment-analysis", model=roberta_model, tokenizer=roberta_tokenizer, device=device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly i

In [49]:
# Function to perform advanced sentiment analysis
def analyze_sentiment(text):
    # RoBERTa sentiment analysis
    roberta_result = sentiment_pipeline(text)[0]
    roberta_sentiment = roberta_result['label']
    roberta_score = roberta_result['score']

    # TextBlob sentiment analysis
    blob = TextBlob(text)
    textblob_polarity = blob.sentiment.polarity
    textblob_subjectivity = blob.sentiment.subjectivity

    # Emotion detection using custom keywords
    emotions = {
        'frustrated': len(re.findall(r'\b(frustrat|confus|difficult)\w*\b', text, re.IGNORECASE)),
        'curious': len(re.findall(r'\b(curious|interest|wonder)\w*\b', text, re.IGNORECASE)),
        'excited': len(re.findall(r'\b(excit|enthusiastic|eager)\w*\b', text, re.IGNORECASE))
    }
    dominant_emotion = max(emotions, key=emotions.get) if any(emotions.values()) else 'neutral'

    # Combine all sentiment analyses
    overall_sentiment = roberta_sentiment.lower()
    if textblob_polarity < -0.3:
        overall_sentiment = 'negative'
    elif textblob_polarity > 0.3:
        overall_sentiment = 'positive'

    return {
        'overall_sentiment': overall_sentiment,
        'roberta_sentiment': roberta_sentiment,
        'roberta_score': roberta_score,
        'textblob_polarity': textblob_polarity,
        'textblob_subjectivity': textblob_subjectivity,
        'dominant_emotion': dominant_emotion
    }

In [50]:
# Function to generate tutoring response based on sentiment
def generate_tutoring_response(query, sentiment_info):
    prompt = f"""As an educational tutor, respond to the following query:
    "{query}"

    The user's sentiment analysis:
    - Overall sentiment: {sentiment_info['overall_sentiment']}
    - Dominant emotion: {sentiment_info['dominant_emotion']}
    - Subjectivity: {sentiment_info['textblob_subjectivity']}

    Adjust your explanation based on this sentiment. If the sentiment is negative or the user seems frustrated,
    provide a simpler explanation with encouragement. If positive or curious, offer a more detailed response.
    If the user seems excited, match their enthusiasm in your response.
    """

    inputs = llama_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = llama_model.generate(**inputs, max_length=300, num_return_sequences=1, temperature=0.7)
    response = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response.strip()

In [51]:
# Function to handle user input and generate response
def tutor_response(user_input, chat_history=None):
    # Initialize chat history if None
    if chat_history is None:
        chat_history = []

    # Analyze the sentiment of the user input
    sentiment_info = analyze_sentiment(user_input)

    # Generate a response based on the sentiment analysis
    response = generate_tutoring_response(user_input, sentiment_info)

    # Sentiment summary (optional display)
    sentiment_summary = f"""Sentiment Analysis:
    - Overall Sentiment: {sentiment_info['overall_sentiment']}
    - Dominant Emotion: {sentiment_info['dominant_emotion']}
    - Subjectivity: {sentiment_info['textblob_subjectivity']:.2f}
    - Confidence: {sentiment_info['roberta_score']:.2f}
    """

    # Append user input and generated response to chat history
    chat_history.append((user_input, response))

    # Create a displayable chat history
    chat_display = ""
    for user_msg, bot_response in chat_history:
        chat_display += f"User: {user_msg}\nTutor: {bot_response}\n\n"

    return chat_display, chat_history

In [52]:
# Create Gradio chat interface
iface = gr.Interface(
    fn=tutor_response,
    inputs=[gr.Textbox(lines=2, placeholder="Ask your question here..."), gr.State()],
    outputs=[gr.Textbox(label="Chat History", lines=20), gr.State()],
    title="Educational Tutor Chatbot with Advanced Sentiment Analysis",
    description="Ask any educational question. The tutor will adjust its response based on your emotional state and the complexity of your query.",
    allow_flagging="never"
)

# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://642ecd4cfb50b07c69.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
